In [89]:
# 1 - Getting data from Web

from bs4 import BeautifulSoup
import pandas as pd
import requests

page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.table
df = pd.read_html(str(table))

data = df[0]
data = data[data['Borough'] != 'Not assigned']
data = data.reset_index(drop=True)
data.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [94]:
# 2 - Adding data from CSV file containing coordinates

data1 = pd.read_csv('Geospatial_Coordinates.csv')
data1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [130]:
#  3 - Uniting data

nb_data = data.join(data1.set_index('Postal Code'), on='Postcode')
tor_ = nb_data[nb_data['Borough'].str.contains('Toronto')].reset_index(drop = True)
a = type(tor_['Latitude'])
tor_['Latitude'] = tor_['Latitude'].astype('float')
tor_['Longitude'] = tor_['Longitude'].astype('float')
tor_.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [128]:
# 4 - Getting Toronto information
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [132]:
# 5 - Creating our map Centered in Toronto

map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(tor_['Latitude'],tor_['Longitude'], tor_['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [145]:
# 6 - LetsStart using Foursquare API

CLIENT_ID = 'QCTDZ0ZUCAKREBPVLLQKIZ5K2DFU2XXB5P33QVNHHK2MHDRP' # your Foursquare ID
CLIENT_SECRET = 'ZHHO4MNOYERHFZJJKQ02ANMKQPXCTAWWYVXKFWX2VXM451R1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QCTDZ0ZUCAKREBPVLLQKIZ5K2DFU2XXB5P33QVNHHK2MHDRP
CLIENT_SECRET:ZHHO4MNOYERHFZJJKQ02ANMKQPXCTAWWYVXKFWX2VXM451R1


In [147]:
# 7 - Getting venues to each Neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [150]:
# 8 - Aplying created function to each row in our  main dataset

tor_venues = getNearbyVenues(names=tor_['Neighbourhood'],
                                   latitudes=tor_['Latitude'],
                                   longitudes=tor_['Longitude']
                                  )
tor_venues.head()

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [158]:
# 9 - Encoding data

#Get Dummies
tor_encoded = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_encoded['Neighbourhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_encoded.columns[-1]] + list(tor_encoded.columns[:-1])
tor_encoded = tor_encoded[fixed_columns]

tor_encoded.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [250]:
# 10 - Calculating Neighborhood caracteristics

tor_grouped = tor_encoded.groupby('Neighbourhood').mean().reset_index()
tor_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,...,0.0,0.0,0.020000,0.0,0.0,0.0,0.01,0.0,0.01,0.000000
1,Bathurst Quay,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.000000
2,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.017857,0.0,0.0,0.0,0.00,0.0,0.00,0.000000
3,Brockton,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.083333
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.055556


In [251]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [252]:
# 11 - Defining top venues in each Neighborhood
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Burger Joint,Hotel,Steakhouse,Restaurant,Asian Restaurant,Cosmetics Shop,Thai Restaurant
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Plane,Boat or Ferry,Bar,Sculpture Garden,Airport Gate
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Steakhouse,Bakery,Farmers Market,Cheese Shop,Café,Italian Restaurant
3,Brockton,Café,Yoga Studio,Breakfast Spot,Coffee Shop,Grocery Store,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection,Gym / Fitness Center
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Butcher,Restaurant,Brewery


In [253]:
# 12 - Lets Cluster!

from sklearn.cluster import KMeans

kclusters = 6

tor_clustering = tor_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#adding cluster information
temp = pd.DataFrame(columns=['Neighbourhood'])
temp['Neighbourhood'] = tor_grouped['Neighbourhood']
temp.insert(0, 'Cluster', kmeans.labels_)
temp.head()

tor = tor_.join(temp.set_index('Neighbourhood'), on='Neighbourhood')

tor.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1
5,M4E,East Toronto,The Beaches,43.676357,-79.293031,5
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,1


In [254]:
# 13 - Recreating a map, now considering the colors of the clusters

import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor['Latitude'], tor['Longitude'], tor['Neighbourhood'], tor['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [260]:
# 14 - Now we can take our own conclusions based on the results and the caracteristics defined. 
tor_a = tor.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [262]:
# In cluster 0, we have Park as the most common vanue, and only one Neighbourhood
tor_a[tor_a['Cluster'] == 0]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,M4N,Central Toronto,Lawrence Park,43.72802,-79.38879,0,Park,Swim School,Bus Line,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [265]:
# In cluster 0, we have a lot of "Coffe shops", "Cafés" and "Gyms", 
#Most of Neighborhoods have this carecteristics
tor_a[tor_a['Cluster'] == 1]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Mexican Restaurant,Café,Farmers Market,Brewery,Shoe Store
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Gym,Park,Fast Food Restaurant,Restaurant,Portuguese Restaurant,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Burger Joint,Tea Room,Bakery,Sporting Goods Shop
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Burger Joint,Tea Room,Bakery,Sporting Goods Shop
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Bakery,Breakfast Spot,Italian Restaurant,Japanese Restaurant,Diner,Park,Cocktail Bar
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Steakhouse,Bakery,Farmers Market,Cheese Shop,Café,Italian Restaurant
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Japanese Restaurant,Burger Joint,Sandwich Place,Juice Bar,Bar,Bakery
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Candy Store,Restaurant,Bank,Baby Store
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,1,Coffee Shop,Café,Bar,Burger Joint,Hotel,Steakhouse,Restaurant,Asian Restaurant,Cosmetics Shop,Thai Restaurant
10,M5H,Downtown Toronto,King,43.650571,-79.384568,1,Coffee Shop,Café,Bar,Burger Joint,Hotel,Steakhouse,Restaurant,Asian Restaurant,Cosmetics Shop,Thai Restaurant


In [266]:
# In cluster 2, we have Garden as the most common vanue, and only one Neighbourhood
tor_a[tor_a['Cluster'] == 2]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Garden,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [268]:
# In cluster 3, we have Arirpor services and, aparently, there is only one 
# Neighbourhood repeated or they are all very close 
tor_a[tor_a['Cluster'] == 2]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,M5V,Downtown Toronto,CN Tower,43.628947,-79.39442,4,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Plane,Boat or Ferry,Bar,Sculpture Garden,Airport Gate
60,M5V,Downtown Toronto,Bathurst Quay,43.628947,-79.39442,4,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Plane,Boat or Ferry,Bar,Sculpture Garden,Airport Gate
61,M5V,Downtown Toronto,Island airport,43.628947,-79.39442,4,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Plane,Boat or Ferry,Bar,Sculpture Garden,Airport Gate
62,M5V,Downtown Toronto,Harbourfront West,43.628947,-79.39442,4,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Plane,Boat or Ferry,Bar,Sculpture Garden,Airport Gate
63,M5V,Downtown Toronto,King and Spadina,43.628947,-79.39442,4,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Plane,Boat or Ferry,Bar,Sculpture Garden,Airport Gate
64,M5V,Downtown Toronto,Railway Lands,43.628947,-79.39442,4,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Plane,Boat or Ferry,Bar,Sculpture Garden,Airport Gate
65,M5V,Downtown Toronto,South Niagara,43.628947,-79.39442,4,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Plane,Boat or Ferry,Bar,Sculpture Garden,Airport Gate


In [269]:
# In cluster 3, we hPlayground as the most common, but,as ocoured reviously, the two 
# have the same coordinates
tor_a[tor_a['Cluster'] == 3]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,M4T,Central Toronto,Moore Park,43.689574,-79.38316,3,Playground,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
50,M4T,Central Toronto,Summerhill East,43.689574,-79.38316,3,Playground,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
